**Here, we are going to replicate the VolGAN-example file, by demonstrating the modifications we are making to the VolGAN file and introducing new functions we have created.**

In [70]:
#import libraries

import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import warnings
import torch.nn as nn
warnings.filterwarnings("ignore")

#import files with supporting functions
import VolGANSwaps as VGS
import Inputs

#import our data sets and a sample date for testing
VOLATILITY_DATA = "data/swaption_atm_vol_full_NEW_no30y.xlsx"
FORWARDS_DATA = "data/forward_sofr_swap_full_NEW_no30y.xlsx"

**Data Loading**
- The data loader takes in the filepath to the volatility surface and the forward swap rates
- Constructs returns on underlying
- Processes the volatility data
    - The surface for every underlying asset is represented as a vector
    - This is similarly the case for the returns
- Sets up the tenor and tau matrices

In [71]:
surfaces_transform, returns, tenor, tau, tenors, taus, dates_dt = VGS.SwapsData(FORWARDS_DATA, VOLATILITY_DATA)

**Data Preprocessing**
- Within `DataPreprocessing` we call `SwapsData` like the original VolGAN implementation
- Here we construct the condition vector, construct the true labels and pass through some other data that might be relevant
    - Each underlying has a condition vector and the true labels (annualized return and log implied vol increment)
    - Because of this they will actually end up being tensor inputs to the models

In [72]:
# Data preprocessing example

true, condition, m_in,sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus = VGS.DataPreprocesssing(FORWARDS_DATA, VOLATILITY_DATA, vol_model='normal')
# true.shape
# condition.shape

**Data Train Test Split**
- Separates dataset into train and test datasets after processing and loading
- Parameterized by datasize percentage / proportion
- Parameterized by a data transformation assumption (`normal` or `log`)
    - Not directly related to volatility model assumptions 

In [73]:
true_train, true_test, condition_train,  condition_test,  m_in,sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus = VGS.DataTrainTest(FORWARDS_DATA, VOLATILITY_DATA, 0.8, vol_model='normal')
# true_train.shape
# true_test.shape
# condition_train.shape
# condition_test.shape

**Next, we are going to run the model.**
- We updated the two training loops in the paper by ensuring the dimensionality of the computations were correct
- We also accounted for the fact that we are using the normal model
- If desired, we can still utilize 

In [74]:
tr = 0.85
noise_dim = 32
hidden_dim = 16
n_epochs = 10000
n_grad = 25
val = True
vol_model = 'normal'
test_epoch = 100

This can be trained on either gpu or cpu.

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We present training for a low number of epochs to ensure that the loops work.
- Output of this volgan function provides us set of data used to train the models
- Also `gen` and `disc` are the trained models that we can then evaluate once trained for more epochs

In [76]:
gen, gen_opt, disc, disc_opt, true_train, true_val, true_test, condition_train, condition_val, condition_test, dates_t,  tenor, tau, tenors, taus  = VGS.VolGAN(FORWARDS_DATA,VOLATILITY_DATA, tr, noise_dim = noise_dim, hidden_dim = hidden_dim, n_epochs = test_epoch,n_grad = n_grad, lrg = 0.0001, lrd = 0.0001, batch_size = 100, device = device, vol_model=vol_model)

 88%|████████▊ | 22/25 [00:07<00:01,  2.92it/s]


KeyboardInterrupt: 

Visualize the results from the Gradient Matching

In [ ]:
tr = 0.85
noise_dim = 32
hidden_dim = 16
n_epochs = 10000
n_grad = 250
val = True
vol_model = 'normal'
lrg = 0.0001 
lrd = 0.0001 
batch_size = 100 

true_train, true_test, condition_train, condition_test,  m_in,sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus = VGS.DataTrainTest(FORWARDS_DATA, VOLATILITY_DATA, tr, vol_model, device)
gen = VGS.Generator(noise_dim=noise_dim,cond_dim=condition_train.shape[2], hidden_dim=hidden_dim,output_dim=true_train.shape[2],mean_in = m_in, std_in = sigma_in, mean_out = m_out, std_out = sigma_out)
gen.to(device)

# m_disc and sigma_disc are not used in the original VolGAN, you can see in the Discriminator forward function 
# we'll preserve the forward pass for now but we can maybe incorporate these later if needed    
m_disc = torch.cat((m_in,m_out),dim=-1)
sigma_disc = torch.cat((sigma_in,sigma_out),dim=-1)

disc = VGS.Discriminator(in_dim = condition_train.shape[2] + true_train.shape[2], hidden_dim = hidden_dim, mean = m_disc, std = sigma_disc)
disc.to(device)

true_val = False
condition_val = False

gen_opt = torch.optim.RMSprop(gen.parameters(), lr=lrg)
disc_opt = torch.optim.RMSprop(disc.parameters(), lr=lrd)

criterion = nn.BCELoss()
criterion = criterion.to(device)

NameError: name 'VOLATILITY_DATA' is not defined

In [ ]:
gen,gen_opt,disc,disc_opt,criterion, alpha, beta = VGS.GradientMatchingPlot(gen,gen_opt,disc,disc_opt,criterion,condition_train,true_train,tenor,tau,tenors,taus,n_grad,lrg,lrd,batch_size,noise_dim,device, vol_model=vol_model)

 13%|█▎        | 33/250 [00:11<01:14,  2.90it/s]


KeyboardInterrupt: 

Full training with 10,000 epochs.

In [ ]:
gen, gen_opt, disc, disc_opt, true_train, true_val, true_test, condition_train, condition_val, condition_test, dates_t,  tenor, tau, tenors, taus  = VGS.VolGAN(FORWARDS_DATA,VOLATILITY_DATA, tr, noise_dim = noise_dim, hidden_dim = hidden_dim, n_epochs = n_epochs,n_grad = n_grad, lrg = 0.0001, lrd = 0.0001, batch_size = 100, device = device, vol_model=vol_model)

100%|██████████| 250/250 [01:26<00:00,  2.90it/s]


alpha : 3.6221621464789517e-06 beta : 0.0005788063762053093


 15%|█▌        | 1508/10000 [08:05<45:35,  3.10it/s] 


KeyboardInterrupt: 

**Saving and Loading Trained Model Parameters**

In [ ]:
# exporting the model

gen_filepath = 'model/generator_parameters.pth'
disc_filepath = 'model/discriminator_parameters.pth'

torch.save(gen.state_dict(), gen_filepath)
torch.save(disc.state_dict(), disc_filepath)

In [ ]:
# loading the model  

tr = 0.85
noise_dim = 32
hidden_dim = 16
n_epochs = 10000
n_grad = 250
val = True
vol_model = 'normal'
lrg = 0.0001 
lrd = 0.0001 
batch_size = 100 

gen = VGS.Generator(noise_dim=noise_dim,cond_dim=condition_train.shape[2], hidden_dim=hidden_dim,output_dim=true_train.shape[2],mean_in = m_in, std_in = sigma_in, mean_out = m_out, std_out = sigma_out)
disc = VGS.Discriminator(in_dim = condition_train.shape[2] + true_train.shape[2], hidden_dim = hidden_dim, mean = m_disc, std = sigma_disc)

gen.to(device)
disc.to(device)

In [ ]:
gen.load_state_dict(torch.load(gen_filepath))
disc.load_state_dict(torch.load(disc_filepath))